PY12 - Pandas advanced

In [ ]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.tail()

In [12]:
melb_df['Date']=pd.to_datetime(melb_df['Date'])
#melb_df['Quarter']=melb_df['Date'].dt.quarter
#melb_df['Quarter'].value_counts()
#melb_df.info()

In [ ]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

In [ ]:
melb_df.sort_values(by=['Price','Distance']).loc[::10, ['Price','Distance']] #show each 10th line of output

Sorting & filtering

In [ ]:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

In [17]:
melb_df.sort_values(by='AreaRatio',ascending=False,ignore_index=True)
melb_df['BuildingArea'].iloc[1558]

126.0

In [18]:
mask1 = melb_df['Rooms'] > 2
mask2 = melb_df['Type'] == 'townhouse'
melb_df[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
)['Price'].iloc[18]

1300000.0

Grouping & aggregation

In [3]:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg('describe')

In [ ]:
melb_df.groupby('Regionname')['Lattitude'].agg(
    ['std']
).sort_values(by='std', ascending=True)

In [ ]:
mask1=melb_df['Date']>'2017-05-01'
mask2=melb_df['Date']<'2017-09-02'
melb_df[mask1 & mask2].groupby('SellerG')['Price'].agg(['sum']).sort_values(by='sum',ascending=False)

Pivot tables

In [22]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


In [ ]:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

In [ ]:
pivot=melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
    fill_value=0
)
#display(pivot)
pivot['unit'].sort_values(ascending=False)

Merging DF

In [52]:
ratings1_df = pd.read_csv('data/ratings1.csv')
ratings2_df = pd.read_csv('data/ratings2.csv')
movies_df = pd.read_csv('data/movies.csv')
dates_df = pd.read_csv('data/dates.csv')
dates_df.head()

,date
0,2000-07-30 18:45:03
1,2000-07-30 18:20:47
2,2000-07-30 18:37:04
3,2000-07-30 19:03:35
4,2000-07-30 18:48:51


In [49]:
dates_df['date']=pd.to_datetime(dates_df['date'])
dates_df['Year']=dates_df['date'].dt.year
dates_df['Year'].mode()

0    2000
Name: Year, dtype: int64

In [56]:
#ratings = pd.concat(
#    [ratings1_df, ratings2_df],
#    ignore_index=True
#)
#ratings = ratings.drop_duplicates(ignore_index=True)
ratings_dates = pd.concat([ratings, dates_df], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


Task 6.3

In [ ]:
import os
import pandas as pd
def concat_user_files(path):
    list=os.listdir(path)
    list.sort()
    list1=[]
    for i in range(len(list)):
        list1.append(pd.read_csv(path+'/'+list[i]))
    users=pd.concat(list1)
    users=users.drop_duplicates(ignore_index=True)
    return users


Task 7.5

In [ ]:
import pandas as pd
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})
purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})
merged = items_df.merge(
    purchase_df,
    on='item_id',
    how='inner'
)
income=sum(merged['stock_count']*merged['price'])